In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import gensim
import os
from konlpy.tag import Mecab
import re
from tqdm import tqdm_notebook
import random
from sklearn.model_selection import train_test_split

print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.4
1.3.3
2.6.0
3.6.5
4.1.2


In [2]:
path = os.getenv("HOME")+'/data/ChatbotData.csv'
corpus = pd.read_csv(path)
corpus

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [3]:
# 데이터의 질문과 답변을 각각 변수에 담아줍니다.
questions = list(x for x in corpus['Q'])
answers = list(x for x in corpus['A'])

In [4]:
def preprocess_sentence(sentence):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ0-9]+", " ", sentence)
    sentence = sentence.strip()
    
    return sentence

In [5]:
mecab = Mecab()
# 문제 : build_corpus를 구축해보세요.
def build_corpus(src, tgt, src_max, tgt_max, src_tok, tgt_tok):
    
    size = len(src)

    src_corpus = []
    tgt_corpus = []

    for idx in tqdm_notebook(range(size)):
        s = src_tok(preprocess_sentence(src[idx]))
        t = tgt_tok(preprocess_sentence(tgt[idx]))
    
        if (len(s) > src_max) | (s in src_corpus): continue
        #[[YOUR CODE]]
        
        src_corpus.append(s)
        tgt_corpus.append(t)

    return src_corpus, tgt_corpus

que_corpus, ans_corpus = \
build_corpus(src=questions,
                tgt=answers,
                src_max=40,
                tgt_max=40,
                src_tok=mecab.morphs,
                tgt_tok=mecab.morphs)

assert len(que_corpus) == len(ans_corpus)

/tmp/ipykernel_84/2423102439.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(size)):


  0%|          | 0/11823 [00:00<?, ?it/s]

In [6]:
word2vec_file_path = os.getenv("HOME")+"/data/word2vec_ko.model"
w2v_ko = gensim.models.Word2Vec.load(word2vec_file_path)

def lexical_sub(tokens):
    import random
    
    res = list()

    try:
        _from = random.choice(tokens)
        _to = w2v_ko.wv.most_similar(_from)[0][0]
        
    except: # 단어장에 없는 단어
        return None

    for tok in tokens:
        if tok is _from: res.append(_to)
        else: res.append(tok)

    return res

In [8]:
from tqdm import tqdm_notebook

print("원본 데이터 크기:", len(que_corpus))

aug_que_corpus = que_corpus.copy()
aug_ans_corpus = ans_corpus.copy()

for idx in tqdm_notebook(range(len(que_corpus))):
    new_que = lexical_sub(que_corpus[idx])

    if new_que is None: continue
    else:
        aug_que_corpus.append(new_que)
        aug_ans_corpus.append(ans_corpus[idx])

# 문제 : que_corpus를 기반으로 tgt corpus를 augmentation해주세요.

    new_ans = lexical_sub(ans_corpus[idx])
    if new_ans is None: continue 
    else:
        aug_que_corpus.append(que_corpus[idx])
        aug_ans_corpus.append(new_ans)


assert len(aug_que_corpus) == len(aug_ans_corpus)

print("최종 데이터 크기:", len(aug_que_corpus))

원본 데이터 크기: 11645


/tmp/ipykernel_84/4124062955.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(que_corpus))):


  0%|          | 0/11645 [00:00<?, ?it/s]

최종 데이터 크기: 34482


In [9]:
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])

['<start>', '12', '시', '땡', '!', '<end>']


In [ ]:
# 문제 : <start> 토큰과 <end> 토큰을 추가해 주세요.
for idx in range(len(aug_ans_corpus)):
    aug_ans_corpus[idx] =#[[YOUR CODE]]

src_corpus = aug_que_corpus
tgt_corpus = aug_ans_corpus

full_corpus = src_corpus + tgt_corpus



enc_train, enc_val, dec_train, dec_val = train_test_split(src_corpus, 
                                                          tgt_corpus, 
                                                          test_size=0.01)

tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
# 문제 : full_corpus로 tokenizer를 학습시켜주세요.
#[[YOUR CODE]]

def tokenize_with_tokenizer(corpus, tokenizer):
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor

enc_train = tokenize_with_tokenizer(src_corpus, tokenizer)
dec_train =#[[YOUR CODE]]

SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = len(tokenizer.word_index) + 1